# Imports

In [253]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd
import numpy as np
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

import numpy as np
from sklearn import datasets
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GroupShuffleSplit
import os
import struct
import datetime
# from skfda import FDataGrid
# from skfda.preprocessing.dim_reduction import FPCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import numpy as np
import pandas as pd
import struct
import matplotlib.pyplot as plt
import torch
from scipy.ndimage import gaussian_filter1d
import torch.nn as nn
import splitfolders
from tensorflow.keras.models import load_model
from scipy.interpolate import CubicSpline
from scipy.signal import find_peaks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import torch
from PIL import Image
from torchvision.transforms import ToTensor
import random
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
# import pytorch libraries
%matplotlib inline
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np4
from sklearn.metrics import r2_score
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score
import torch
import torch.distributed
import pandas as pd
import numpy as np
from ABRA_35 import interpolate_and_smooth, CNN, plot_wave, calculate_and_plot_wave, plot_waves_single_frequency, arfread, get_str, calculate_hearing_threshold, all_thresholds, peak_finding
import warnings
from sklearn.preprocessing import StandardScaler,MinMaxScaler
warnings.filterwarnings('ignore')
import os
import io
import re
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [317]:
def CV_modeltesting(K, X, y, mod):
    kf = KFold(n_splits=K)

    train_fold_RMSES = []
    test_fold_RMSES = []

    for i, (train_index, test_index) in enumerate(kf.split(X)):

        print(f'Processing Fold {i+1}')
        X_train, y_train = X.iloc[train_index,:], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index,:], y.iloc[test_index]

        y_preds_train = mod.predict(X_train)
        RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))
        print(f'Train RMSE = {RMSE_train}')
        train_fold_RMSES.append(RMSE_train)

        y_preds_test = mod.predict(X_test)
        RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))
        print(f'Test RMSE = {RMSE_test}\n')
        test_fold_RMSES.append(RMSE_test)

    avg_train_RMSE = np.mean(train_fold_RMSES)
    avg_test_RMSE = np.mean(test_fold_RMSES)

    print(f'Average Train RMSE across {K} folds: {avg_train_RMSE}')
    print(f'Average Test RMSE across {K} folds: {avg_test_RMSE}')

    return avg_train_RMSE, avg_test_RMSE

In [ ]:
def stratified_kfold_by_feature(X, y, model, stratify_col, n_splits=5, random_state=42):
    """
    Perform cross-validation stratified by a specific feature column,
    but exclude that column from being used as a predictor.
    
    Parameters:
    -----------
    X : DataFrame
        The feature matrix (must be a pandas DataFrame)
    y : Series or array-like
        The target variable
    model : estimator object
        The model to evaluate
    stratify_col : str
        Name of the column in X to stratify by (will be excluded from predictors)
    n_splits : int, default=5
        Number of folds
    random_state : int, default=42
        Random state for reproducibility
        
    Returns:
    --------
    train_rmse_scores : list
        RMSE scores for training sets
    test_rmse_scores : list
        RMSE scores for test sets
    """
    # Ensure X is a DataFrame
    if not isinstance(X, pd.DataFrame):
        raise TypeError("X must be a pandas DataFrame to use column name for stratification")
    
    # Extract the stratification column
    stratify_values = X[stratify_col].values
    
    # Initialize StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Lists to store the RMSE scores for each fold
    train_rmse_scores = []
    test_rmse_scores = []
    
    # Remove the stratify column from the features
    X_for_model = X.drop(columns=[stratify_col])
    
    # Iterate through each fold
    for i, (train_index, test_index) in enumerate(skf.split(X, stratify_values)):
        print(f'Processing Fold {i+1}')
        
        # Split the data
        X_train, X_test = X_for_model.iloc[train_index], X_for_model.iloc[test_index]
        
        if isinstance(y, pd.Series):
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        else:
            y_train, y_test = y[train_index], y[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        # Calculate RMSE
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        
        # Store the scores
        train_rmse_scores.append(train_rmse)
        test_rmse_scores.append(test_rmse)
        
        # Print fold statistics with stratification distribution
        train_strat_dist = X.iloc[train_index][stratify_col].value_counts(normalize=True).to_dict()
        test_strat_dist = X.iloc[test_index][stratify_col].value_counts(normalize=True).to_dict()
        
        print(f'Fold {i+1} - Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}')
        print(f'  Train distribution of {stratify_col}: {train_strat_dist}')
        print(f'  Test distribution of {stratify_col}: {test_strat_dist}')
    
    # Calculate and print average scores
    avg_train_rmse = np.mean(train_rmse_scores)
    avg_test_rmse = np.mean(test_rmse_scores)
    
    print(f'\nAverage Train RMSE: {avg_train_rmse:.4f}')
    print(f'Average Test RMSE: {avg_test_rmse:.4f}')
    
    return train_rmse_scores, test_rmse_scores

# Data Processing

In [ ]:
filter1 = 128
filter2 = 32
dropout1 = 0.5
dropout2 = 0.3
dropout_fc = 0.1

# Model initialization
peak_finding_model = CNN(filter1, filter2, dropout1, dropout2, dropout_fc)
model_loader = torch.load('./models/waveI_cnn.pth')
peak_finding_model.load_state_dict(model_loader)
peak_finding_model.eval()

def peak_finding(wave):
    # Prepare waveform
    waveform=interpolate_and_smooth(wave) # Added indexing per calculate and plot wave function
    # waveform_torch = torch.tensor(waveform, dtype=torch.float32).unsqueeze(0) archived ABRA
    waveform_torch = torch.tensor(waveform, dtype=torch.float32).unsqueeze(0).unsqueeze(0) #newer ABRA
    # print(waveform_torch)
    # Get prediction from model
    outputs = peak_finding_model(waveform_torch)
    prediction = int(round(outputs.detach().numpy()[0][0], 0))
    # prediction_test = int(round(outputs.detach().numpy()[0], 0))
    # print("Model output:", outputs, "Prediction true start:", prediction)

    # Apply Gaussian smoothing
    smoothed_waveform = gaussian_filter1d(waveform, sigma=1)

    # Find peaks and troughs
    n = 18
    t = 14
    # start_point = prediction - 9 archived ABRA
    start_point = prediction - 6 #newer ABRA
    smoothed_peaks, _ = find_peaks(smoothed_waveform[start_point:], distance=n)
    smoothed_troughs, _ = find_peaks(-smoothed_waveform, distance=t)
    sorted_indices = np.argsort(smoothed_waveform[smoothed_peaks+start_point])
    highest_smoothed_peaks = np.sort(smoothed_peaks[sorted_indices[-5:]] + start_point)
    relevant_troughs = np.array([])
    for p in range(len(highest_smoothed_peaks)):
        c = 0
        for t in smoothed_troughs:
            if t > highest_smoothed_peaks[p]:
                if p != 4:
                    try:
                        if t < highest_smoothed_peaks[p+1]:
                            relevant_troughs = np.append(relevant_troughs, int(t))
                            break
                    except IndexError:
                        pass
                else:
                    relevant_troughs = np.append(relevant_troughs, int(t))
                    break
    relevant_troughs = relevant_troughs.astype('i')
    return highest_smoothed_peaks, relevant_troughs

def extract_metadata(metadata_lines):
    # Dictionary to store extracted metadata
    metadata = {}
    
    for line in metadata_lines:
        # Extract SW FREQ
        freq_match = re.search(r'SW FREQ:\s*(\d+\.?\d*)', line)
        if freq_match:
            metadata['SW_FREQ'] = float(freq_match.group(1))
        
        # Extract LEVELS
        levels_match = re.search(r':LEVELS:\s*([^:]+)', line)
        if levels_match:
            # Split levels and convert to list of floats
            metadata['LEVELS'] = [float(level) for level in levels_match.group(1).split(';') if level]
    
    return metadata

def read_custom_tsv(file_path):
    # Read the entire file
    with open(file_path, 'r', encoding='ISO-8859-1') as f:
        content = f.read()
    
    # Split the content into metadata and data sections
    metadata_lines = []
    data_section = None
    
    # Find the ':DATA' marker
    data_start = content.find(':DATA')
    
    if data_start != -1:
        # Extract metadata (lines before ':DATA')
        metadata_lines = content[:data_start].split('\n')
        
        # Extract data section
        data_section = content[data_start:].split(':DATA')[1].strip()
    
    # Extract specific metadata
    metadata = extract_metadata(metadata_lines)
    
    # Read the data section directly
    try:
        # Use StringIO to create a file-like object from the data section
        raw_data = pd.read_csv(
            io.StringIO(data_section), 
            sep='\s+',  # Use whitespace as separator
            header=None
        )
        raw_data = raw_data.T
        # Add metadata columns to the DataFrame
        if 'SW_FREQ' in metadata:
            raw_data['Freq(kHz)'] = metadata['SW_FREQ']
            # raw_data['Freq(Hz)'] = raw_data['Freq(Hz)'].apply(lambda x: x*1000)
        
        if 'LEVELS' in metadata:
            # Repeat levels to match the number of rows
            levels_repeated = metadata['LEVELS'] * (len(raw_data) // len(metadata['LEVELS']) + 1)
            raw_data['Level(dB)'] = levels_repeated[:len(raw_data)]
        
        filtered_data = raw_data.apply(pd.to_numeric, errors='coerce').dropna()
        filtered_data.columns = filtered_data.columns.map(str)

        columns = ['Freq(kHz)'] + ['Level(dB)'] + [col for col in filtered_data.columns if col.isnumeric() == True]
        filtered_data = filtered_data[columns]
        return filtered_data
    
    except Exception as e:
        print(f"Error reading data: {e}")
        return None, metadata


def peaks_troughs_amp_final(df, freq, db, time_scale=10, multiply_y_factor=1.0, units='Microvolts'):
    db_column = 'Level(dB)'
    
    khz = df[(df['Freq(kHz)'] == freq) & (df[db_column] == db)]
    if not khz.empty:
        index = khz.index.values[0]
        final = df.loc[index, '0':].dropna()
        final = pd.to_numeric(final, errors='coerce').dropna()

        target = int(244 * (time_scale / 10))
        
        # Process the wave as in calculate_and_plot_wave
        y_values = interpolate_and_smooth(final, target)
        
        # Apply scaling factor
        y_values *= multiply_y_factor
        
        # Handle units conversion if needed
        if units == 'Nanovolts':
            y_values /= 1000
            
        # Generate normalized version for peak finding
        y_values_fpf = interpolate_and_smooth(y_values[:244])
        
        # Standardize and normalize for peak finding, exactly as in the original
        flattened_data = y_values_fpf.flatten().reshape(-1, 1)
        scaler = StandardScaler()
        standardized_data = scaler.fit_transform(flattened_data)
        min_max_scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = min_max_scaler.fit_transform(standardized_data).reshape(y_values_fpf.shape)
        y_values_fpf = interpolate_and_smooth(scaled_data[:244])
        
        # Find peaks using the normalized data
        highest_peaks, relevant_troughs = peak_finding(y_values_fpf)
        
        # Calculate amplitude on the processed but non-normalized data
        if highest_peaks.size > 0 and relevant_troughs.size > 0:
            # Following the same approach as in the display_metrics_table function
            first_peak_amplitude = y_values[highest_peaks[0]] - y_values[relevant_troughs[0]]
            return highest_peaks, relevant_troughs, first_peak_amplitude
    
    return None, None, None

In [ ]:
#ORIGINAL DATA SPLIT

time_scale = 18
amp_per_freq = {'Subject': [], 'Freq(kHz) (x1)': [], 'Level(dB) (x2)': [], 'Amplitude (x3)':[]}
start_path = '/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/abr_data/WPZ Electrophysiology'
for subject in os.listdir(start_path):
    # print("Subject:",subject)
    for fq in os.listdir(os.path.join(start_path,subject)):
        # print(fq)
        if fq.startswith('ABR') and fq.endswith('.tsv'):
            path = os.path.join(start_path,subject,fq)
            data_df = read_custom_tsv(path)
            # print(data_df)
            freqs = data_df['Freq(kHz)'].unique().tolist()
            levels = data_df['Level(dB)'].unique().tolist()
            for freq in freqs:
                for lvl in levels:
                    # print("Frequency=",freq, "Level=", lvl)
                    _, _, amp = peaks_troughs_amp_final(df=data_df, freq=freq, db=lvl, time_scale=time_scale)
                    # print(f'Amplitude: {amp}\n')
                    amp_per_freq['Subject'].append(subject)
                    amp_per_freq['Freq(kHz) (x1)'].append(freq)
                    amp_per_freq['Level(dB) (x2)'].append(lvl)
                    amp_per_freq['Amplitude (x3)'].append(amp)
        else:
            pass

amp_df_full = pd.DataFrame(data=amp_per_freq)

raw_synapse_counts = pd.read_excel('/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/WPZ Ribbon and Synapse Counts.xlsx')
raw_synapse_counts = raw_synapse_counts.mask(lambda x: x.isnull()).dropna()
raw_synapse_counts['Synapses to IHC (y1)'] = raw_synapse_counts.iloc[:,6]
raw_synapse_counts['vx (x4)'] = raw_synapse_counts['vx']
raw_synapse_counts.drop(columns=['vx'], inplace=True)
raw_synapse_counts.rename(columns={'Freq':'Freq(kHz) (x1)'}, inplace=True)
# raw_synapse_counts['Freq(Hz) (x1)'] = raw_synapse_counts['Freq(Hz) (x1)'].apply(lambda x: x*1000) # PUTTING BACK
raw_synapse_counts.rename(columns={'Case':'Subject', 'IHCs' : 'IHCs (y2)'}, inplace=True)

paired = amp_df_full.join(raw_synapse_counts.set_index(['Subject', 'Freq(kHz) (x1)']), on=['Subject', 'Freq(kHz) (x1)'])
slice = paired[paired['Subject']=='WPZ174'][['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)','Synapses to IHC (y1)', 'IHCs (y2)']]
final = paired[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)','Synapses to IHC (y1)', 'IHCs (y2)']]
final_clean = final.dropna()

# adding in the strain feature
strains = pd.read_excel('/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/WPZ Mouse groups.xlsx')
final_clean_strained = final_clean.join(strains.set_index('ID#'), on='Subject')
final_clean_strained['Strain'] = final_clean_strained['Strain'].str.strip()
final_clean_strained = final_clean_strained.rename(columns={'Strain': 'Strain (x5)'})
final_clean_strained = final_clean_strained.dropna()
final_clean_strained_foundation = final_clean_strained.copy()
final_clean_strained = final_clean_strained[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)', 'Strain (x5)', 'Synapses to IHC (y1)', 'Group']]
np.unique(final_clean_strained['Group'])

final_clean_70 = final_clean[final_clean['Level(dB) (x2)'] >= 70.0]
final_clean_strained_70 = final_clean_strained[final_clean_strained['Level(dB) (x2)'] >= 70.0]
# np.unique(final_clean['Level(dB) (x2)']) max level is 80 db
len(final_clean), len(final_clean_70) # 10000 less data points!!!

final_clean_strained_grouped = final_clean_strained.copy()
final_clean_strained_grouped['Group - dB'] = final_clean_strained_grouped['Group'].apply(lambda x: x.split(' ')[0] if x.split(' ')[0].endswith('dB') else 'Control')
final_clean_strained_grouped['Group - Time Elapsed'] = final_clean_strained_grouped['Group'].apply(lambda x: x.split(' ')[1] if x.split(' ')[1].endswith(('h', 'wks', 'w')) else x.split(' ')[0])
final_clean_strained_grouped.head()

final_clean_strained_grouped_pos = final_clean_strained_grouped.copy()
final_clean_strained_grouped_pos['Amplitude (x3)'] = final_clean_strained_grouped['Amplitude (x3)'].apply(lambda x: 0 if x < 0 else x)

len(final_clean_strained_grouped_pos[final_clean_strained_grouped_pos['Amplitude (x3)'] < 0])

final_clean_strained_grouped_pos['Amplitude (x3)'] = final_clean_strained_grouped['Amplitude (x3)'].apply(lambda x: 0 if x < 0 else x)

# final_clean_strained_grouped_pos[(final_clean_strained_grouped_pos['Subject'] == 'WPZ66') & (final_clean_strained_grouped_pos['Amplitude (x3)'] ==0.055901451434921576)
final_clean_strained_grouped_pos_cleangroup = final_clean_strained_grouped_pos.copy()
final_clean_strained_grouped_pos_cleangroup['Group'] = final_clean_strained_grouped_pos_cleangroup['Group'].apply(lambda x: x.strip())
np.unique(final_clean_strained_grouped_pos_cleangroup['Group'])

final_clean_strained_grouped_pos_cleangroup.head()
final_clean_strained_grouped_pos_cleangroup_vs = final_clean_strained_grouped_pos_cleangroup.copy()
final_clean_strained_grouped_pos_cleangroup_vs['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs['Group - dB']
# final_clean_strained_grouped_pos_cleangroup_vs['Group - Time Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs['Group - Time Elapsed']
final_clean_strained_grouped_pos_cleangroup_vs = final_clean_strained_grouped_pos_cleangroup_vs[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)',
       'vx (x4)', 'Strain (x5)','Group - dB (x6)', 'Group - Time Elapsed', 'Group','Synapses to IHC (y1)']]

def split_on_number(input_string):
    return re.findall(r"[A-Za-z]+|\d+", input_string)

hrs_week = 24*7

final_clean_strained_grouped_pos_cleangroup_vs_timed = final_clean_strained_grouped_pos_cleangroup_vs.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(lambda x: '0dB' if x == 'Control' else x)
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(split_on_number)
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(lambda x: x[0])
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(lambda x: int(x.strip()))
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Split'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed'].apply(split_on_number)
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Magn.'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Split'].apply(lambda x: x[0])
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Magn.'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Magn.'].apply(lambda x: int(x.strip()))
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Unit'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Split'].apply(lambda x: x[1])
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Unit'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Unit'].apply(lambda x: "wks" if x == 'w' else x)
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Hours Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed.apply(lambda row: row['Group - Time Elapsed - Magn.']* hrs_week if row['Group - Time Elapsed - Unit'] == 'wks' else row['Group - Time Elapsed - Magn.'], axis = 1)

# final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Days Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Hours Elapsed (x7)'].apply(lambda x: x/24)
# final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(lambda )

In [128]:
# NEW DATA SPLIT

time_scale = 18
amp_per_freq = {'Subject': [], 'Freq(kHz) (x1)': [], 'Level(dB) (x2)': [], 'Amplitude (x3)':[]}
start_path = '/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/abr_data/WPZ Electrophysiology'
for subject in os.listdir(start_path):
    # print("Subject:",subject)
    for fq in os.listdir(os.path.join(start_path,subject)):
        # print(fq)
        if fq.startswith('ABR') and fq.endswith('.tsv'):
            path = os.path.join(start_path,subject,fq)
            data_df = read_custom_tsv(path)
            # print(data_df)
            freqs = data_df['Freq(kHz)'].unique().tolist()
            levels = data_df['Level(dB)'].unique().tolist()
            for freq in freqs:
                for lvl in levels:
                    # print("Frequency=",freq, "Level=", lvl)
                    _, _, amp = peaks_troughs_amp_final(df=data_df, freq=freq, db=lvl, time_scale=time_scale)
                    # print(f'Amplitude: {amp}\n')
                    amp_per_freq['Subject'].append(subject)
                    amp_per_freq['Freq(kHz) (x1)'].append(freq)
                    amp_per_freq['Level(dB) (x2)'].append(lvl)
                    amp_per_freq['Amplitude (x3)'].append(amp)
        else:
            pass

amp_df_full = pd.DataFrame(data=amp_per_freq)

raw_synapse_counts = pd.read_excel('/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/WPZ Ribbon and Synapse Counts.xlsx')
raw_synapse_counts = raw_synapse_counts.mask(lambda x: x.isnull()).dropna()
raw_synapse_counts['Synapses to IHC (y1)'] = raw_synapse_counts.iloc[:,6]
raw_synapse_counts['vx (x4)'] = raw_synapse_counts['vx']
raw_synapse_counts.drop(columns=['vx'], inplace=True)
raw_synapse_counts.rename(columns={'Freq':'Freq(kHz) (x1)'}, inplace=True)
# raw_synapse_counts['Freq(Hz) (x1)'] = raw_synapse_counts['Freq(Hz) (x1)'].apply(lambda x: x*1000) # PUTTING BACK
raw_synapse_counts.rename(columns={'Case':'Subject'}, inplace=True)

paired2 = amp_df_full.join(raw_synapse_counts.set_index(['Subject', 'Freq(kHz) (x1)']), on=['Subject', 'Freq(kHz) (x1)'])
# lilslice = paired[paired['Subject']=='WPZ174'][['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)','Synapses to IHC (y1)', 'IHCs']]
final2 = paired2[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)','Synapses to IHC (y1)', 'Synapses', 'IHCs']]
final_clean2 = final2.dropna()

# adding in the strain feature
strains = pd.read_excel('/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/WPZ Mouse groups.xlsx')
final_clean_strained2 = final_clean2.join(strains.set_index('ID#'), on='Subject')
final_clean_strained2['Strain'] = final_clean_strained2['Strain'].str.strip()
final_clean_strained2 = final_clean_strained2.rename(columns={'Strain': 'Strain (x5)'})
final_clean_strained2 = final_clean_strained2.dropna()
final_clean_strained2 = final_clean_strained2[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)', 'Strain (x5)', 'Synapses to IHC (y1)', 'Group', 'Synapses', 'IHCs']]
# np.unique(final_clean_strained2['Group'])

# final_clean_70 = final_clean[final_clean['Level(dB) (x2)'] >= 70.0]
# final_clean_strained_70 = final_clean_strained[final_clean_strained['Level(dB) (x2)'] >= 70.0]
# # np.unique(final_clean['Level(dB) (x2)']) max level is 80 db
# len(final_clean), len(final_clean_70) # 10000 less data points!!!

final_clean_strained_grouped2 = final_clean_strained2.copy()
final_clean_strained_grouped2['Group - dB'] = final_clean_strained_grouped2['Group'].apply(lambda x: x.split(' ')[0] if x.split(' ')[0].endswith('dB') else 'Control')
final_clean_strained_grouped2['Group - Time Elapsed'] = final_clean_strained_grouped2['Group'].apply(lambda x: x.split(' ')[1] if x.split(' ')[1].endswith(('h', 'wks', 'w')) else x.split(' ')[0])
final_clean_strained_grouped2.head()

final_clean_strained_grouped_pos2 = final_clean_strained_grouped2.copy()
final_clean_strained_grouped_pos2['Amplitude (x3)'] = final_clean_strained_grouped2['Amplitude (x3)'].apply(lambda x: 0 if x < 0 else x)

len(final_clean_strained_grouped_pos2[final_clean_strained_grouped_pos2['Amplitude (x3)'] < 0])

final_clean_strained_grouped_pos2['Amplitude (x3)'] = final_clean_strained_grouped2['Amplitude (x3)'].apply(lambda x: 0 if x < 0 else x)

# final_clean_strained_grouped_pos[(final_clean_strained_grouped_pos['Subject'] == 'WPZ66') & (final_clean_strained_grouped_pos['Amplitude (x3)'] ==0.055901451434921576)
final_clean_strained_grouped_pos_cleangroup2 = final_clean_strained_grouped_pos2.copy()
final_clean_strained_grouped_pos_cleangroup2['Group'] = final_clean_strained_grouped_pos_cleangroup2['Group'].apply(lambda x: x.strip())
np.unique(final_clean_strained_grouped_pos_cleangroup2['Group'])

final_clean_strained_grouped_pos_cleangroup2.head()
final_clean_strained_grouped_pos_cleangroup_vs2 = final_clean_strained_grouped_pos_cleangroup2.copy()
final_clean_strained_grouped_pos_cleangroup_vs2['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs2['Group - dB']
# final_clean_strained_grouped_pos_cleangroup_vs['Group - Time Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs['Group - Time Elapsed']
final_clean_strained_grouped_pos_cleangroup_vs2 = final_clean_strained_grouped_pos_cleangroup_vs2[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)',
       'vx (x4)', 'Strain (x5)','Group - dB (x6)', 'Group - Time Elapsed', 'Group','Synapses to IHC (y1)', 'Synapses', 'IHCs']]

def split_on_number(input_string):
    return re.findall(r"[A-Za-z]+|\d+", input_string)

hrs_week = 24*7

final_clean_strained_grouped_pos_cleangroup_vs_timed2 = final_clean_strained_grouped_pos_cleangroup_vs2.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'].apply(lambda x: '0dB' if x == 'Control' else x)
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'].apply(split_on_number)
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'].apply(lambda x: x[0])
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'].apply(lambda x: int(x.strip()))
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Split'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed'].apply(split_on_number)
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Magn.'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Split'].apply(lambda x: x[0])
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Magn.'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Magn.'].apply(lambda x: int(x.strip()))
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Unit'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Split'].apply(lambda x: x[1])
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Unit'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Unit'].apply(lambda x: "wks" if x == 'w' else x)
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Hours Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2.apply(lambda row: row['Group - Time Elapsed - Magn.']* hrs_week if row['Group - Time Elapsed - Unit'] == 'wks' else row['Group - Time Elapsed - Magn.'], axis = 1)
final_clean_strained_grouped_pos_cleangroup_vs_timed2
# final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Days Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Hours Elapsed (x7)'].apply(lambda x: x/24)
# final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(lambda )

,Subject,Freq(kHz) (x1),Level(dB) (x2),Amplitude (x3),vx (x4),Strain (x5),Group - dB (x6),Group - Time Elapsed,Group,Synapses to IHC (y1),Synapses,IHCs,Group - Time Elapsed - Split,Group - Time Elapsed - Magn.,Group - Time Elapsed - Unit,Group - Hours Elapsed (x7)
0,WPZ145,45.2,70.0,0.033579,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344
0,WPZ145,45.2,70.0,0.033579,v2,C57B6,98,8wks,98dB 8wks post,10.888889,98.0,9,"[8, wks]",8,wks,1344
1,WPZ145,45.2,75.0,0.034262,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344
1,WPZ145,45.2,75.0,0.034262,v2,C57B6,98,8wks,98dB 8wks post,10.888889,98.0,9,"[8, wks]",8,wks,1344
2,WPZ145,45.2,80.0,0.154224,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7328,WPZ101,32.0,60.0,1.634279,v2,C57B6,0,8wks,8wks ctrl,15.463918,150.0,9.7,"[8, wks]",8,wks,1344
7329,WPZ101,32.0,70.0,1.769193,v1,C57B6,0,8wks,8wks ctrl,16.923077,154.0,9.1,"[8, wks]",8,wks,1344
7329,WPZ101,32.0,70.0,1.769193,v2,C57B6,0,8wks,8wks ctrl,15.463918,150.0,9.7,"[8, wks]",8,wks,1344
7330,WPZ101,32.0,80.0,1.023154,v1,C57B6,0,8wks,8wks ctrl,16.923077,154.0,9.1,"[8, wks]",8,wks,1344


In [126]:
final_clean2

,Subject,Freq(kHz) (x1),Level(dB) (x2),Amplitude (x3),vx (x4),Synapses to IHC (y1),IHCs (y2)
0,WPZ145,45.2,70.0,0.033579,v1,8.750000,8.8
0,WPZ145,45.2,70.0,0.033579,v2,10.888889,9
1,WPZ145,45.2,75.0,0.034262,v1,8.750000,8.8
1,WPZ145,45.2,75.0,0.034262,v2,10.888889,9
2,WPZ145,45.2,80.0,0.154224,v1,8.750000,8.8
...,...,...,...,...,...,...,...
7328,WPZ101,32.0,60.0,1.634279,v2,15.463918,9.7
7329,WPZ101,32.0,70.0,1.769193,v1,16.923077,9.1
7329,WPZ101,32.0,70.0,1.769193,v2,15.463918,9.7
7330,WPZ101,32.0,80.0,1.023154,v1,16.923077,9.1


# Vanilla Lin Reg Model

In [73]:
X = final_clean_strained_grouped_pos_cleangroup_vs_timed[['Freq(kHz) (x1)', 'Level(dB) (x2)','Amplitude (x3)']]
y = final_clean_strained_grouped_pos_cleangroup_vs_timed['Synapses to IHC (y1)']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.2, random_state=42)

In [74]:
model_vanilla = LinearRegression().fit(X_train,y_train)

y_preds_train = model.predict(X_train)
RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))

y_preds_test = model.predict(X_test)
RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))

RMSE_train, RMSE_test

(2.8962291261784285, 2.895859167040614)

In [76]:
# Cross validation
K=5
kf = KFold(n_splits=K)

train_fold_RMSES = []
test_fold_RMSES = []

for i, (train_index, test_index) in enumerate(kf.split(X)):

    print(f'Processing Fold {i+1}')
    X_train, y_train = X.iloc[train_index,:], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index,:], y.iloc[test_index]

    y_preds_train = model.predict(X_train)
    RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))
    print(f'Train RMSE = {RMSE_train}')
    train_fold_RMSES.append(RMSE_train)

    y_preds_test = model.predict(X_test)
    RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))
    print(f'Test RMSE = {RMSE_test}\n')
    test_fold_RMSES.append(RMSE_test)

avg_train_RMSE = np.mean(train_fold_RMSES)
avg_test_RMSE = np.mean(test_fold_RMSES)

print(f'Average Train RMSE across {K} folds: {avg_train_RMSE}')
print(f'Average Test RMSE across {K} folds: {avg_test_RMSE}')

Processing Fold 1
Train RMSE = 2.913936454557555
Test RMSE = 2.8239328735809175

Processing Fold 2
Train RMSE = 2.98472400080733
Test RMSE = 2.5109567657958833

Processing Fold 3
Train RMSE = 2.8023395289110073
Test RMSE = 3.2444697246817285

Processing Fold 4
Train RMSE = 2.9618158381505606
Test RMSE = 2.617029707707716

Processing Fold 5
Train RMSE = 2.8131074445144497
Test RMSE = 3.2069717077308724

Average Train RMSE across 5 folds: 2.8951846533881804
Average Test RMSE across 5 folds: 2.8806721558994233


## Sign Tests

In [286]:
final_clean_strained_grouped_pos_cleangroup_vs_timed_ols = final_clean_strained_grouped_pos_cleangroup_vs_timed.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed_ols = final_clean_strained_grouped_pos_cleangroup_vs_timed_ols.rename(columns = {'Freq(kHz) (x1)' : 'FreqkHz', 'Level(dB) (x2)': 'LeveldB', 'Amplitude (x3)' : 'Amplitude', 'Synapses to IHC (y1)' : 'Synapses_to_IHC'})
final_clean_strained_grouped_pos_cleangroup_vs_timed_ols.columns

Index(['Subject', 'FreqkHz', 'LeveldB', 'Amplitude', 'vx (x4)', 'Strain (x5)',
       'Group - dB (x6)', 'Group - Time Elapsed', 'Group', 'Synapses_to_IHC',
       'Group - Time Elapsed - Split', 'Group - Time Elapsed - Magn.',
       'Group - Time Elapsed - Unit', 'Group - Hours Elapsed (x7)'],
      dtype='object')

In [289]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.api as sm

# Full model
full_model = ols('Synapses_to_IHC ~ FreqkHz + LeveldB + Amplitude', 
                data=final_clean_strained_grouped_pos_cleangroup_vs_timed_ols).fit()
print("Full Model Summary:")
print(full_model.summary())

# Partial F-test for Amplitude
reduced_model1 = ols('Synapses_to_IHC ~ FreqkHz + LeveldB', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed_ols).fit()
print("\nPartial F-test for Amplitude:")
print(anova_lm(reduced_model1, full_model))

# Partial F-test for FreqkHz
reduced_model2 = ols('Synapses_to_IHC ~ LeveldB + Amplitude', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed_ols).fit()
print("\nPartial F-test for FreqkHz:")
print(anova_lm(reduced_model2, full_model))

# Partial F-test for LeveldB
reduced_model3 = ols('Synapses_to_IHC ~ FreqkHz + Amplitude', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed_ols).fit()
print("\nPartial F-test for LeveldB:")
print(anova_lm(reduced_model3, full_model))

# Partial F-test for FreqkHz and LeveldB jointly
reduced_model4 = ols('Synapses_to_IHC ~ Amplitude', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed_ols).fit()
print("\nPartial F-test for FreqkHz and LeveldB together:")
print(anova_lm(reduced_model4, full_model))

Full Model Summary:
                            OLS Regression Results                            
Dep. Variable:        Synapses_to_IHC   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     389.0
Date:                Thu, 27 Mar 2025   Prob (F-statistic):          2.75e-241
Time:                        16:57:15   Log-Likelihood:                -30252.
No. Observations:               12187   AIC:                         6.051e+04
Df Residuals:                   12183   BIC:                         6.054e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     16.9318      0.076

# Stratified CV

In [101]:
np.unique(final_clean_strained_grouped_pos_cleangroup_vs_timed['Strain (x5)'])

array(['C57B6', 'CBA/CaJ'], dtype=object)

In [102]:
final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded = final_clean_strained_grouped_pos_cleangroup_vs_timed.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded['Strain (x5)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded['Strain (x5)'].apply(lambda x: 0 if x == 'C57B6' else 1)

np.unique(final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded['Strain (x5)'])

array([0, 1])

In [103]:
X = final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded[['Freq(kHz) (x1)', 'Level(dB) (x2)','Amplitude (x3)', 'Strain (x5)']]
y = final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded['Synapses to IHC (y1)']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.2, random_state=42, stratify=final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded['Strain (x5)'])


In [104]:
model_strain_strat = LinearRegression().fit(X_train,y_train)

y_preds_train = model_strain_strat.predict(X_train)
RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))

y_preds_test = model_strain_strat.predict(X_test)
RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))

RMSE_train, RMSE_test

(2.8396751633699444, 2.8321396400722816)

In [233]:
train_rmses, test_rmses = stratified_kfold_by_feature(X, y, model_strain_strat, stratify_col='Strain (x5)', n_splits=5)

Processing Fold 1
Fold 1 - Train RMSE: 2.8593, Test RMSE: 2.7928
  Train distribution of Strain (x5): {0: 0.8502410503641399, 1: 0.1497589496358601}
  Test distribution of Strain (x5): {0: 0.850287120590648, 1: 0.14971287940935193}
Processing Fold 2
Fold 2 - Train RMSE: 2.8486, Test RMSE: 2.8358
  Train distribution of Strain (x5): {0: 0.8502410503641399, 1: 0.1497589496358601}
  Test distribution of Strain (x5): {0: 0.850287120590648, 1: 0.14971287940935193}
Processing Fold 3
Fold 3 - Train RMSE: 2.8546, Test RMSE: 2.8118
  Train distribution of Strain (x5): {0: 0.8502564102564103, 1: 0.14974358974358976}
  Test distribution of Strain (x5): {0: 0.850225687320476, 1: 0.149774312679524}
Processing Fold 4
Fold 4 - Train RMSE: 2.8024, Test RMSE: 3.0167
  Train distribution of Strain (x5): {0: 0.8502564102564103, 1: 0.14974358974358976}
  Test distribution of Strain (x5): {0: 0.850225687320476, 1: 0.149774312679524}
Processing Fold 5
Fold 5 - Train RMSE: 2.8627, Test RMSE: 2.7774
  Train d

In [423]:
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx 

,Subject,Freq(kHz) (x1),Level(dB) (x2),Amplitude (x3),vx (x4),Strain (x5),Group - dB (x6),Group - Time Elapsed,Group,Synapses to IHC (y1),Synapses,IHCs,Group - Time Elapsed - Split,Group - Time Elapsed - Magn.,Group - Time Elapsed - Unit,Group - Hours Elapsed (x7),CrossFreq_SIHCRatio (y2)
0,WPZ145,45.2,70.0,0.033579,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344,9.831461
0,WPZ145,45.2,70.0,0.033579,v2,C57B6,98,8wks,98dB 8wks post,10.888889,98.0,9,"[8, wks]",8,wks,1344,9.831461
1,WPZ145,45.2,75.0,0.034262,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344,9.831461
1,WPZ145,45.2,75.0,0.034262,v2,C57B6,98,8wks,98dB 8wks post,10.888889,98.0,9,"[8, wks]",8,wks,1344,9.831461
2,WPZ145,45.2,80.0,0.154224,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344,9.831461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7328,WPZ101,32.0,60.0,1.634279,v2,C57B6,0,8wks,8wks ctrl,15.463918,150.0,9.7,"[8, wks]",8,wks,1344,16.170213
7329,WPZ101,32.0,70.0,1.769193,v1,C57B6,0,8wks,8wks ctrl,16.923077,154.0,9.1,"[8, wks]",8,wks,1344,16.170213
7329,WPZ101,32.0,70.0,1.769193,v2,C57B6,0,8wks,8wks ctrl,15.463918,150.0,9.7,"[8, wks]",8,wks,1344,16.170213
7330,WPZ101,32.0,80.0,1.023154,v1,C57B6,0,8wks,8wks ctrl,16.923077,154.0,9.1,"[8, wks]",8,wks,1344,16.170213


# Predicting on the Average between Viewing Fields

In [427]:
freqs = np.unique(final_clean_strained_grouped_pos_cleangroup_vs_timed2['Freq(kHz) (x1)'])
subs = np.unique(final_clean_strained_grouped_pos_cleangroup_vs_timed2['Subject'])
# final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['CrossFreq_SIHCRatio (y2)'] = 0
for freq in freqs:
    for sub in subs:
        mask = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx[(final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Freq(kHz) (x1)'] == freq) & (final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Subject'] == sub)] # global for updates
        if len(mask) > 0:

            mask1 = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx[(final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Freq(kHz) (x1)'] == freq) & (final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Subject'] == sub) & (final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['vx (x4)'] == 'v1')]
            mask2 = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx[(final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Freq(kHz) (x1)'] == freq) & (final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Subject'] == sub) & (final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['vx (x4)'] == 'v2')]

            if not mask1.empty and not mask2.empty:
                mask1 = mask1.reset_index().iloc[0,:]
                mask2 = mask2.reset_index().iloc[0,:]

                total_syns = float(mask1['Synapses'] + mask2['Synapses'])
                total_ihcs = float(mask1['IHCs'] + mask2['IHCs'])
                ratio = total_syns / total_ihcs
                # print(total_syns, total_ihcs)
                # if total_syns == 0.0 or total_ihcs == 0.0:
                #     print(sub, freq)
                mask_index = mask.index
                final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx.loc[mask_index,'CrossFreq_SIHCRatio (y2)'] = ratio

final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['CrossFreq_SIHCRatio (y2)']

0        9.831461
0        9.831461
1        9.831461
1        9.831461
2        9.831461
          ...    
7328    16.170213
7329    16.170213
7329    16.170213
7330    16.170213
7330    16.170213
Name: CrossFreq_SIHCRatio (y2), Length: 12187, dtype: float64

In [426]:
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx

,Subject,Freq(kHz) (x1),Level(dB) (x2),Amplitude (x3),vx (x4),Strain (x5),Group - dB (x6),Group - Time Elapsed,Group,Synapses to IHC (y1),Synapses,IHCs,Group - Time Elapsed - Split,Group - Time Elapsed - Magn.,Group - Time Elapsed - Unit,Group - Hours Elapsed (x7),CrossFreq_SIHCRatio (y2)
0,WPZ145,45.2,70.0,0.033579,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344,9.831461
0,WPZ145,45.2,70.0,0.033579,v2,C57B6,98,8wks,98dB 8wks post,10.888889,98.0,9,"[8, wks]",8,wks,1344,9.831461
1,WPZ145,45.2,75.0,0.034262,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344,9.831461
1,WPZ145,45.2,75.0,0.034262,v2,C57B6,98,8wks,98dB 8wks post,10.888889,98.0,9,"[8, wks]",8,wks,1344,9.831461
2,WPZ145,45.2,80.0,0.154224,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344,9.831461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7328,WPZ101,32.0,60.0,1.634279,v2,C57B6,0,8wks,8wks ctrl,15.463918,150.0,9.7,"[8, wks]",8,wks,1344,16.170213
7329,WPZ101,32.0,70.0,1.769193,v1,C57B6,0,8wks,8wks ctrl,16.923077,154.0,9.1,"[8, wks]",8,wks,1344,16.170213
7329,WPZ101,32.0,70.0,1.769193,v2,C57B6,0,8wks,8wks ctrl,15.463918,150.0,9.7,"[8, wks]",8,wks,1344,16.170213
7330,WPZ101,32.0,80.0,1.023154,v1,C57B6,0,8wks,8wks ctrl,16.923077,154.0,9.1,"[8, wks]",8,wks,1344,16.170213


In [221]:
X = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx[['Freq(kHz) (x1)', 'Level(dB) (x2)','Amplitude (x3)']]
y = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['CrossFreq_SIHCRatio (y2)']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.2, random_state=42)

In [ ]:
model_avg = LinearRegression().fit(X_train,y_train)

y_preds_train = model_avg.predict(X_train)
RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))

y_preds_test = model_avg.predict(X_test)
RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))

RMSE_train, RMSE_test

(2.8550383868161306, 2.809730708641642)

In [ ]:
CV_modeltesting(5, X, y)

Processing Fold 1
Train RMSE = 2.886838782352169
Test RMSE = 2.6770189314300614

Processing Fold 2
Train RMSE = 2.8975812792576545
Test RMSE = 2.6301996813955

Processing Fold 3
Train RMSE = 2.7386335902853784
Test RMSE = 3.240606795032601

Processing Fold 4
Train RMSE = 2.926657181804546
Test RMSE = 2.49796415741554

Processing Fold 5
Train RMSE = 2.776022020542586
Test RMSE = 3.1107275550318314

Average Train RMSE across 5 folds: 2.8451465708484664
Average Test RMSE across 5 folds: 2.831303424061107


(2.8451465708484664, 2.831303424061107)

# CV and strat with average

In [240]:
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['Strain (x5)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Strain (x5)'].apply(lambda x: 0 if x == 'C57B6' else 1)

np.unique(final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded['Strain (x5)'])

array([0, 1])

In [273]:
X = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded[['Freq(kHz) (x1)', 'Level(dB) (x2)','Amplitude (x3)', 'Strain (x5)']]
y = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['CrossFreq_SIHCRatio (y2)']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.2, random_state=42, stratify=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['Strain (x5)'])

model_avg_strat = LinearRegression().fit(X_train,y_train)

y_preds_train = model_avg_strat.predict(X_train)
RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))

y_preds_test = model_avg_strat.predict(X_test)
RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))

RMSE_train, RMSE_test

(2.782994962533814, 2.784551261309025)

In [246]:
stratified_kfold_by_feature(X, y, model_avg_strat, 'Strain (x5)', n_splits=5, random_state=42)

Processing Fold 1
Fold 1 - Train RMSE: 2.8593, Test RMSE: 2.7928
  Train distribution of Strain (x5): {0: 0.8502410503641399, 1: 0.1497589496358601}
  Test distribution of Strain (x5): {0: 0.850287120590648, 1: 0.14971287940935193}
Processing Fold 2
Fold 2 - Train RMSE: 2.8486, Test RMSE: 2.8358
  Train distribution of Strain (x5): {0: 0.8502410503641399, 1: 0.1497589496358601}
  Test distribution of Strain (x5): {0: 0.850287120590648, 1: 0.14971287940935193}
Processing Fold 3
Fold 3 - Train RMSE: 2.8546, Test RMSE: 2.8118
  Train distribution of Strain (x5): {0: 0.8502564102564103, 1: 0.14974358974358976}
  Test distribution of Strain (x5): {0: 0.850225687320476, 1: 0.149774312679524}
Processing Fold 4
Fold 4 - Train RMSE: 2.8024, Test RMSE: 3.0167
  Train distribution of Strain (x5): {0: 0.8502564102564103, 1: 0.14974358974358976}
  Test distribution of Strain (x5): {0: 0.850225687320476, 1: 0.149774312679524}
Processing Fold 5
Fold 5 - Train RMSE: 2.8627, Test RMSE: 2.7774
  Train d

([2.8593100269612495,
  2.8485848543093146,
  2.8545606174936444,
  2.802416986249799,
  2.8627248637946963],
 [2.792816529817402,
  2.8358183967258204,
  2.8117558719870215,
  3.0166896322711962,
  2.7774033198878034])

# Significance Tests - Avg Model

In [278]:
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols.rename(columns = {'Freq(kHz) (x1)' : 'FreqkHz', 'Level(dB) (x2)': 'LeveldB', 'Amplitude (x3)' : 'Amplitude', 'CrossFreq_SIHCRatio (y2)' : 'CrossFreqSIHCRatio'})
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols.columns

Index(['Subject', 'FreqkHz', 'LeveldB', 'Amplitude', 'vx (x4)', 'Strain (x5)',
       'Group - dB (x6)', 'Group - Time Elapsed', 'Group',
       'Synapses to IHC (y1)', 'Synapses', 'IHCs',
       'Group - Time Elapsed - Split', 'Group - Time Elapsed - Magn.',
       'Group - Time Elapsed - Unit', 'Group - Hours Elapsed (x7)',
       'CrossFreqSIHCRatio'],
      dtype='object')

In [ ]:
model = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + FreqkHz*Amplitude', data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
FreqkHz,875.532365,1.0,110.750892,8.655803e-26
LeveldB,5866.837509,1.0,742.128459,1.112180e-158
Amplitude,4293.831720,1.0,543.150328,1.423835e-117
FreqkHz:Amplitude,2203.380128,1.0,278.717639,6.950048e-62
FreqkHz:LeveldB,1440.081882,1.0,182.163857,3.228899e-41
Residual,96295.926702,12181.0,NaN,NaN


In [380]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.api as sm

# Full model
full_model = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + FreqkHz*Amplitude', 
                data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
print("Full Model Summary:")
print(full_model.summary())

# Partial F-test for Amplitude
# reduced_model1 = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for Amplitude:")
# print(anova_lm(reduced_model1, full_model))

# # Partial F-test for FreqkHz
# reduced_model2 = ols('CrossFreqSIHCRatio ~ LeveldB + Amplitude', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for FreqkHz:")
# print(anova_lm(reduced_model2, full_model))

# # Partial F-test for LeveldB
# reduced_model3 = ols('CrossFreqSIHCRatio ~ FreqkHz + Amplitude', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for LeveldB:")
# print(anova_lm(reduced_model3, full_model))

# # Partial F-test for FreqkHz and LeveldB jointly
# reduced_model4 = ols('CrossFreqSIHCRatio ~ Amplitude', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for FreqkHz and LeveldB together:")
# print(anova_lm(reduced_model4, full_model))

# Testing interactions 

#  + FreqkHz*LeveldB + LeveldB*Amplitude + FreqkHz*Amplitude + FreqkHz*LeveldB*Amplitude



# reduced_model5 = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + FreqkHz*LeveldB + FreqkHz*Amplitude', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for FreqkHz*LeveldB:")
# print(anova_lm(reduced_model5, full_model))

# Partial F-test for FreqkHz
# reduced_model6 = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + FreqkHz*Amplitude', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for FreqkHz*Amplitude:")
# print(anova_lm(reduced_model2, full_model))

# # Partial F-test for LeveldB
# reduced_model7 = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + LeveldB*Amplitude', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for LeveldB*Amplitude:")
# print(anova_lm(reduced_model3, full_model))

# # # Partial F-test for FreqkHz and LeveldB jointly
# reduced_model8 = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for FreqkHz and LeveldB together:")
# print(anova_lm(reduced_model4, full_model))

Full Model Summary:
                            OLS Regression Results                            
Dep. Variable:     CrossFreqSIHCRatio   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     329.4
Date:                Mon, 31 Mar 2025   Prob (F-statistic):          1.34e-269
Time:                        15:23:17   Log-Likelihood:                -29979.
No. Observations:               12187   AIC:                         5.997e+04
Df Residuals:                   12182   BIC:                         6.000e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         

# With freq*amp

In [344]:
X = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols[['FreqkHz', 'LeveldB','Amplitude', 'Strain (x5)', 'Group']]
y = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols['CrossFreqSIHCRatio']

In [322]:
import statsmodels.formula.api as smf

In [328]:
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols.head()

,Subject,FreqkHz,LeveldB,Amplitude,vx (x4),Strain (x5),Group - dB (x6),Group - Time Elapsed,Group,Synapses to IHC (y1),Synapses,IHCs,Group - Time Elapsed - Split,Group - Time Elapsed - Magn.,Group - Time Elapsed - Unit,Group - Hours Elapsed (x7),CrossFreqSIHCRatio
0,WPZ145,45.2,70.0,0.033579,v1,0,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344,9.831461
0,WPZ145,45.2,70.0,0.033579,v2,0,98,8wks,98dB 8wks post,10.888889,98.0,9,"[8, wks]",8,wks,1344,9.831461
1,WPZ145,45.2,75.0,0.034262,v1,0,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344,9.831461
1,WPZ145,45.2,75.0,0.034262,v2,0,98,8wks,98dB 8wks post,10.888889,98.0,9,"[8, wks]",8,wks,1344,9.831461
2,WPZ145,45.2,80.0,0.154224,v1,0,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344,9.831461


In [420]:
model_int = smf.ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + FreqkHz*Amplitude', data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# stratified_kfold_by_feature(X, y, model_int, 'Strain (x5)', n_splits=5, random_state=42)
# CV_modeltesting(5, X, y, model_int)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.2)

y_preds_train = model_int.predict(X_train)
RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))

y_preds_test = model_int.predict(X_test)
RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))

RMSE_train, RMSE_test

(2.8213365750621615, 2.873784141654663)

In [375]:
stratified_kfold_by_feature2(X, y, model_int, 'Group')

Processing Fold 1
Fold 1 - Train RMSE: 2.8287, Test RMSE: 2.7368
  Train distribution of Group: {'6wks ctrl': 0.14062980818545492, '14wks ctrl': 0.12667965945225151, '98dB 8wks post': 0.09754846650938558, '90dB 2w post': 0.08667555646732998, '8wks ctrl': 0.08503436249871782, '94dB 2w post': 0.062057646938147505, '98dB 2w post': 0.05826238588573187, '94dB 0h post': 0.05744178890142579, '94dB 8wks post': 0.056928915786234484, '98dB 24h post': 0.053031080110780594, '90dB 0h post': 0.053031080110780594, '94dB 24h post': 0.05221048312647451, '90dB 24h post': 0.050466714534824084, '101dB 24h post': 0.020002051492460766}
  Test distribution of Group: {'6wks ctrl': 0.14068908941755537, '14wks ctrl': 0.12674323215750616, '98dB 8wks post': 0.09762100082034454, '90dB 2w post': 0.08654634946677604, '8wks ctrl': 0.08490566037735849, '94dB 2w post': 0.061936013125512716, '98dB 2w post': 0.05824446267432322, '94dB 0h post': 0.05742411812961444, '94dB 8wks post': 0.05701394585726005, '98dB 24h post': 

([2.8287136871275824,
  2.8155763433358207,
  2.809250135919753,
  2.7890722306611653,
  2.8100705978885823],
 [2.7367912453828285,
  2.7904370609530638,
  2.815820598029498,
  2.8949590704698958,
  2.8125433509149347])

In [376]:
stratified_kfold_by_feature2(X, y, model_int, 'Strain (x5)')

Processing Fold 1
Fold 1 - Train RMSE: 2.8235, Test RMSE: 2.7583
  Train distribution of Strain (x5): {0: 0.8502410503641399, 1: 0.1497589496358601}
  Test distribution of Strain (x5): {0: 0.850287120590648, 1: 0.14971287940935193}
Processing Fold 2
Fold 2 - Train RMSE: 2.8088, Test RMSE: 2.8175
  Train distribution of Strain (x5): {0: 0.8502410503641399, 1: 0.1497589496358601}
  Test distribution of Strain (x5): {0: 0.850287120590648, 1: 0.14971287940935193}
Processing Fold 3
Fold 3 - Train RMSE: 2.8250, Test RMSE: 2.7520
  Train distribution of Strain (x5): {0: 0.8502564102564103, 1: 0.14974358974358976}
  Test distribution of Strain (x5): {0: 0.850225687320476, 1: 0.149774312679524}
Processing Fold 4
Fold 4 - Train RMSE: 2.7673, Test RMSE: 2.9772
  Train distribution of Strain (x5): {0: 0.8502564102564103, 1: 0.14974358974358976}
  Test distribution of Strain (x5): {0: 0.850225687320476, 1: 0.149774312679524}
Processing Fold 5
Fold 5 - Train RMSE: 2.8277, Test RMSE: 2.7409
  Train d

([2.8234788077137085,
  2.80881593531172,
  2.8250115368521143,
  2.7673493917444105,
  2.8277197453215988],
 [2.758322696157662,
  2.817549458258851,
  2.7520096896458464,
  2.977195338631362,
  2.7408592939706202])

In [403]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler

X = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols[['FreqkHz', 'LeveldB','Amplitude', 'Strain (x5)']]
y = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols['CrossFreqSIHCRatio']

# Daten vorbereiten - standardisieren ist wichtig für Regularisierung
X_scaled = StandardScaler().fit_transform(X[['FreqkHz', 'LeveldB','Amplitude']])

# Ridge-Regression mit verschiedenen Regularisierungsstärken
alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
ridge_results = []

for alpha in alphas:
    # Mit regularisierten statsmodels OLS
    model_ridge = smf.ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + FreqkHz*Amplitude', 
                         data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit_regularized(L1_wt=0.0, alpha=alpha)
    
    # Train-Test-Split wie vorher
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        shuffle=True, 
        test_size=0.2, 
        stratify=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols['Strain (x5)'],
        random_state=42
    )
    
    # Vorhersagen und RMSE berechnen
    y_preds_train = model_ridge.predict(X_train)
    RMSE_train = np.sqrt(mean_squared_error(y_train, y_preds_train))
    
    y_preds_test = model_ridge.predict(X_test)
    RMSE_test = np.sqrt(mean_squared_error(y_test, y_preds_test))
    
    ridge_results.append({
        'alpha': alpha,
        'RMSE_train': RMSE_train,
        'RMSE_test': RMSE_test,
        'diff': RMSE_train - RMSE_test
    })

In [404]:
ridge_results

[{'alpha': 0.001,
  'RMSE_train': 2.8300652992332345,
  'RMSE_test': 2.8417651465838083,
  'diff': -0.011699847350573833},
 {'alpha': 0.01,
  'RMSE_train': 2.869685328178581,
  'RMSE_test': 2.887291022297049,
  'diff': -0.01760569411846813},
 {'alpha': 0.1,
  'RMSE_train': 3.900941232106652,
  'RMSE_test': 3.9685173627374537,
  'diff': -0.06757613063080159},
 {'alpha': 1.0,
  'RMSE_train': 5.881305979750584,
  'RMSE_test': 6.00584515970534,
  'diff': -0.12453917995475638},
 {'alpha': 10.0,
  'RMSE_train': 6.355989594772559,
  'RMSE_test': 6.493336021560671,
  'diff': -0.1373464267881115},
 {'alpha': 100.0,
  'RMSE_train': 6.5009866657548585,
  'RMSE_test': 6.646428492689638,
  'diff': -0.14544182693477925}]

In [335]:
def stratified_kfold_by_feature2(X, y, model, stratify_col, n_splits=5, random_state=42):
    """
    Perform cross-validation stratified by a specific feature column,
    but exclude that column from being used as a predictor.
    
    Parameters:
    -----------
    X : DataFrame
        The feature matrix (must be a pandas DataFrame)
    y : Series or array-like
        The target variable
    model : estimator object
        The model to evaluate
    stratify_col : str
        Name of the column in X to stratify by (will be excluded from predictors)
    n_splits : int, default=5
        Number of folds
    random_state : int, default=42
        Random state for reproducibility
        
    Returns:
    --------
    train_rmse_scores : list
        RMSE scores for training sets
    test_rmse_scores : list
        RMSE scores for test sets
    """
    # Ensure X is a DataFrame
    if not isinstance(X, pd.DataFrame):
        raise TypeError("X must be a pandas DataFrame to use column name for stratification")
    
    # Extract the stratification column
    stratify_values = X[stratify_col].values
    
    # Initialize StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Lists to store the RMSE scores for each fold
    train_rmse_scores = []
    test_rmse_scores = []
    
    # Remove the stratify column from the features
    X_for_model = X.drop(columns=[stratify_col])
    
    # Iterate through each fold
    for i, (train_index, test_index) in enumerate(skf.split(X, stratify_values)):
        print(f'Processing Fold {i+1}')
        
        # Split the data
        X_train, X_test = X_for_model.iloc[train_index], X_for_model.iloc[test_index]
        
        if isinstance(y, pd.Series):
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        else:
            y_train, y_test = y[train_index], y[test_index]
        
        # Train the model
        # model.fit(X_train, y_train)
        
        # Make predictions
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        # Calculate RMSE
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        
        # Store the scores
        train_rmse_scores.append(train_rmse)
        test_rmse_scores.append(test_rmse)
        
        # Print fold statistics with stratification distribution
        train_strat_dist = X.iloc[train_index][stratify_col].value_counts(normalize=True).to_dict()
        test_strat_dist = X.iloc[test_index][stratify_col].value_counts(normalize=True).to_dict()
        
        print(f'Fold {i+1} - Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}')
        print(f'  Train distribution of {stratify_col}: {train_strat_dist}')
        print(f'  Test distribution of {stratify_col}: {test_strat_dist}')
    
    # Calculate and print average scores
    avg_train_rmse = np.mean(train_rmse_scores)
    avg_test_rmse = np.mean(test_rmse_scores)
    
    print(f'\nAverage Train RMSE: {avg_train_rmse:.4f}')
    print(f'Average Test RMSE: {avg_test_rmse:.4f}')
    
    return train_rmse_scores, test_rmse_scores

In [ ]:
# def CV_modeltesting(K, X, y, mod):
#     kf = KFold(n_splits=K)

#     train_fold_RMSES = []
#     test_fold_RMSES = []

#     for i, (train_index, test_index) in enumerate(kf.split(X)):

#         print(f'Processing Fold {i+1}')
#         X_train, y_train = X.iloc[train_index,:], y.iloc[train_index]
#         X_test, y_test = X.iloc[test_index,:], y.iloc[test_index]

#         y_preds_train = mod.predict(X_train)
#         RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))
#         print(f'Train RMSE = {RMSE_train}')
#         train_fold_RMSES.append(RMSE_train)

#         y_preds_test = mod.predict(X_test)
#         RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))
#         print(f'Test RMSE = {RMSE_test}\n')
#         test_fold_RMSES.append(RMSE_test)

#     avg_train_RMSE = np.mean(train_fold_RMSES)
#     avg_test_RMSE = np.mean(test_fold_RMSES)

#     print(f'Average Train RMSE across {K} folds: {avg_train_RMSE}')
#     print(f'Average Test RMSE across {K} folds: {avg_test_RMSE}')

#     return avg_train_RMSE, avg_test_RMSE